# Отчет

Василиадис Янис ИПОВС-12

**Цель работы**

Найти оптимальное расположение скважин для максимизации добычи нефти

**Описание задачи**

Для добычи нефти требуется проводить бурение эксплуатационных скважин.
Целью задачи построения оптимальной системы разработки является
определение положения скважин для максимизации NPV.
При дальнейшем моделировании издержки на бурение и ввод в эксплуатацию
скважин, текущие и налоговые издержки не учитываем.
В результате, главным является максимизировать итоговую добычу.


**Используемые технологии**

* Kotlin
* TornadoFX
* Лень
* Мозг

**Железо**
* i3-3120M 2.50GHz x 2(4)
* DDR3-1600 12GB
* SSD Samsung 850 EVO

**План**

* Теоретическая часть

* Описание реализации оптимизационных алгоритмов

* Тестирование отдельных элементов кода

* Тестирование на реальных данных

## Теоретическая часть + Описание реализации оптимизационных алгоритмов

Решение задачи было разделено на следующие составляющие:

1. Модуль для работы с файлами
2. Подсчет площади 
3. Оптимизация 
4. Интерфейс
5. График

### Модуль работы с файлами

Считывание больших файлов и преобразование их в необходимый формат(объект) является важной задачей, так как размеры файлов могут доходить до ~1G для поля 10 000 000 X 10 000 000, к примеру. Основной проблемой стала скорость считывания. Если использовать такую функцию преоьзраваония строки в массив по пробелу, то считывание файла в ~500MB происходит около 2-3 минут. Воспользовавших информацией о размерности входных данных (см. формат файла), было принято решение читать "окном" по целым строкам с распаралеливанием парсинга этих строк. В этом случае считывание происходит за ~8-10 сек. 

Код который производит считывание даных ячеек и преобразовывает их в массив обьектов `IndexFloat`. Последний представляет собой число `Float` и ссылку на список:

`
class IndexFloat(var value: Float) {
    var subMap: BooleanArray = booleanArrayOf()
}
`

`
val arrayData = data[3]
        .split(" ")
        .map { IndexFloat(it.toDouble()) }
        .windowed(sizeX, sizeY)
        .toCollection(ArrayList(sizeX * sizeY))
`


Так же для удобства считанные файлы сериализовались (объект языка сохраняется в последовательность байтов), для того, что бы при следующем запуске программы, если карта не изменилась, можно было бы без парсинга сразу считать объект карты. В среднем считывание обьекта происходит за ~1 сек.




## Подсчет площади

Задача подсчета площади усложняется тем, что необходимо было учитывать пересечения областей (окружностей) вышек, а так как таких пересечений может быть N, это исключает аналитический способ подсчета. Из рассмотренных алгоритмов: Quadtree Approximation, Monte Carlo Estimation, An Exact Solution [(подробнее)](https://www.benfrederickson.com/calculating-the-intersection-of-3-or-more-circles/). Первые два способа крайне точны, но очень затратны, последний сложен в написании. Так как сроки были сжаты и высокая точность не была нужна, было принято решение использовать следующий лагоритм (самописное кривое нечто):

Имеющияся данные:

* Список окружностей
* Карта (сетка с коэфицентами в каждой ячейке)

Класс окружности содержит в себе координаты и поле для хранения площади.


`
class MyCircleData {
    var x: Float = 0f
    var y: Float = 0f
    var r: Float = 0f
    var calculatedArea: Double = 0.0
}
`

1. берем первую окружность
2. вычисляем границы окружности (описывающий квадрат)
3. ходим по полученному квадрату и выполняем следующие действия
    1. вычисляем расстояние от всех вершин квадрата до центра окружности и сравниваем с радиусом, если окружность пересекает ячейку, то продолжаем работу дальше, если нет, то берем следующую ячейку.
    2. выполняем проверку на то, что текущая ячейка вся пренадлежит одной окружности, если так, то вычисляем ее площадь и идем дальше, если нет то выполняем следующие шаги:
        1. разбиваем текущую ячейку на более мелкую сетку, размером 1х1
        2. итерируемся по этому разбиению и вычисляем расстояние от левого верхнего угла квадратика до центра всех окружностей, если квадратик пренадлежит окружности, то вычисляется количество окружностей, коэфицент ячейки делится на это число и прибавляется всем пересикающим окружностям в `calculatedArea`.

        
